In [1]:
import csv
from collections import defaultdict

# NOME DO ARQUIVO CSV ESPERADO NO MESMO DIRETÓRIO DO SCRIPT
NOME_ARQUIVO_CSV = "interacoes_globo.csv"

def carregar_dados_de_arquivo_csv(nome_arquivo):
    """
    Carrega os dados de um arquivo CSV para uma lista de listas.
    A primeira lista retornada é o cabeçalho, e as subsequentes são as linhas de dados.
    Retorna None se o arquivo não for encontrado ou ocorrer um erro.
    """
    dados_com_cabecalho = []
    try:
        with open(nome_arquivo, mode='r', encoding='utf-8', newline='') as arquivo_csv:
            leitor_csv = csv.reader(arquivo_csv)
            # Adiciona todas as linhas (incluindo o cabeçalho) à lista
            for linha in leitor_csv:
                dados_com_cabecalho.append(linha)

        if not dados_com_cabecalho:
            print(f"Aviso: O arquivo CSV '{nome_arquivo}' está vazio.")
            return None
        return dados_com_cabecalho
    except FileNotFoundError:
        print(f"Erro: Arquivo '{nome_arquivo}' não encontrado. Certifique-se de que ele está na mesma pasta do script.")
        return None
    except Exception as e:
        print(f"Erro ao ler o arquivo CSV '{nome_arquivo}': {e}")
        return None

def converter_lista_para_lista_de_dicionarios(dados_em_lista_com_cabecalho):
    """
    Converte uma lista de listas (onde a primeira lista é o cabeçalho)
    para uma lista de dicionários.
    """
    cabecalho = dados_em_lista_com_cabecalho[0]  # Extrai a primeira linha como cabeçalho.
    linhas = dados_em_lista_com_cabecalho[1:]  # As demais são os dados.

    lista_de_dicionarios = []  # Inicializa a lista final.
    for linha in linhas:
        if len(linha) != len(cabecalho):  # Ignora linhas com número incorreto de colunas.
            continue
        dicionario = {chave: valor for chave, valor in zip(cabecalho, linha)}  # Cria dicionário usando zip().
        lista_de_dicionarios.append(dicionario)  # Adiciona à lista.
    return lista_de_dicionarios #Retorna a lista dos dicionarios

def tratar_campos_inteiros(interacao_bruta, interacao_limpa):

    """
    Trata os campos que devem ser inteiros simples (id_conteudo e id_usuario), convertendo-os e tratando erros.
    Retorna a interacao limpa com os campos convertidos.
    """
    try:
        interacao_limpa['id_conteudo'] = int(interacao_bruta.get('id_conteudo', -1)) # Tenta converter o campo 'id_conteudo' para inteiro
    except:
        interacao_limpa['id_conteudo'] = -1 # Se falhar, define como -1

    try:
        interacao_limpa['id_usuario'] = int(interacao_bruta.get('id_usuario', -1)) # Tenta converter o campo 'id_usuario' para inteiro
    except:
        interacao_limpa['id_usuario'] = -1 # Se falhar, define como -1

    return interacao_limpa # Retorna o dicionário com os campos tratados

def tratar_watch_duration_seconds(interacao_bruta, interacao_limpa):

    """
    Trata o campo watch_duration_seconds, convertendo-o e tratando erros.
    Se o campo estiver vazio ou ausente, deve ser definido como 0.
    Retorna a interacao limpa com o campo watch_duration_seconds convertido.
    """
    tipo = interacao_bruta.get('tipo_interacao', '').strip().lower()  # Obtém e normaliza tipo_interacao.
    valor = interacao_bruta.get('watch_duration_seconds', '').strip()  # Obtém valor de duração assistida.

    if valor == '':
        interacao_limpa['watch_duration_seconds'] = 0  # Se vazio, define como 0.
    else:
        try:
            interacao_limpa['watch_duration_seconds'] = int(valor)  # Converte para inteiro.
        except ValueError:
            interacao_limpa['watch_duration_seconds'] = 0  # Se falhar, define como 0.

    return interacao_limpa  # Retorna o dicionário atualizado.

def tratar_campos_texto(interacao_bruta, interacao_limpa):
    """
    Trata os campos de texto, removendo espaços extras e convertendo para string.
    Retorna a interacao limpa com os campos convertidos.
    """
    campos = ['nome_conteudo', 'timestamp_interacao', 'plataforma', 'tipo_interacao', 'comment_text']
    for campo in campos:
        valor = interacao_bruta.get(campo, '')  # Obtém o valor do campo.
        interacao_limpa[campo] = valor.strip() if valor else ''  # Remove espaços extras.
    return interacao_limpa  # Retorna o dicionário atualizado.

def limpar_e_transformar_dados(lista_interacoes_brutas_dict):
    """
    Limpa e transforma os dados brutos das interações (lista de dicionários).
    Converte tipos de dados, trata valores ausentes e remove espaços.
    essa função deve usar as 3 funcoes acima
    Retorna a lista de interações limpas (também como lista de dicionários).
    """

    interacoes_limpas = [] #Cria uma lista vazia para as interacoes limpas
    for interacao_bruta in lista_interacoes_brutas_dict: # Para cada interação bruta na lista
        interacao_limpa = {} # Cria um dicionário vazio para a interação limpa
        interacao_limpa = tratar_campos_inteiros(interacao_bruta, interacao_limpa) # Trata campos inteiros e adiciona ao dicionário limpo
        interacao_limpa = tratar_watch_duration_seconds(interacao_bruta, interacao_limpa) # Trata o tempo de visualização e adiciona ao dicionário limpo
        interacao_limpa = tratar_campos_texto(interacao_bruta, interacao_limpa)  # Trata campos de texto e adiciona ao dicionário limpo
        interacoes_limpas.append(interacao_limpa) # Adiciona a interação limpa na lista final
    return interacoes_limpas # Retorna a lista com todas as interações limpas

def criar_mapa_conteudos(interacoes_limpas):
    """
    Cria um dicionário que mapeia id_conteudo para nome_conteudo.
    """
    mapa = {}  # Inicializa o dicionário
    for interacao in interacoes_limpas:
        mapa[interacao['id_conteudo']] = interacao['nome_conteudo']  # Associa id ao nome
    return mapa  # Retorna o dicionário de mapeamento

def calcular_metricas_por_conteudo(interacoes_limpas, mapa_conteudos):
    """
    Calcula várias métricas de engajamento agrupadas por conteúdo.
    Aqui você deve varrer as interações limpas e calcular as métricas desejadas.
    Recomenda-se utilizar um loop e verificar se o conteudo ja foi incluido pelo nome ou id_conteudo
    Retorna um dicionário com as métricas calculadas.
    """
    metricas_conteudo = defaultdict(lambda: {
        'nome_conteudo': '',
        'total_interacoes_engajamento': 0, # like, share, comment, vote_bbb
        'contagem_por_tipo_interacao': defaultdict(int),
        'tempo_total_visualizacao': 0,
        'soma_watch_duration_para_media': 0,
        'contagem_watch_duration_para_media': 0,
        'media_tempo_visualizacao': 0.0,
        'comentarios': []
    })

    tipos_engajamento = {'like', 'share', 'comment', 'vote_bbb'} #Tipos de Engajamento

    for interacao in interacoes_limpas:
        id_c = interacao['id_conteudo']  # ID do conteúdo
        metricas_c = metricas_conteudo[id_c]  # Obtém entrada para esse conteúdo

        if not metricas_c['nome_conteudo']:  # Se o nome ainda não foi preenchido
            metricas_c['nome_conteudo'] = mapa_conteudos.get(id_c, "Desconhecido")  # Pega nome do mapeamento

        # Métrica 1: Total de interações de engajamento
        if interacao['tipo_interacao'] in tipos_engajamento:
            metricas_c['total_interacoes_engajamento'] += 1  # Soma interações de engajamento

        # Métrica 2: Contagem de cada tipo_interacao
        metricas_c['contagem_por_tipo_interacao'][interacao['tipo_interacao']] += 1

        # Métrica 3: Tempo total de watch_duration_seconds
        metricas_c['tempo_total_visualizacao'] += interacao['watch_duration_seconds']

        # Métrica 4: Média de watch_duration_seconds (considerar apenas watch_duration_seconds > 0)
        if interacao['watch_duration_seconds'] > 0:
            metricas_c['soma_watch_duration_para_media'] += interacao['watch_duration_seconds']
            metricas_c['contagem_watch_duration_para_media'] += 1

        # Métrica 5: Listar todos os comentários
        if interacao['tipo_interacao'] == 'comment' and interacao['comment_text']:
            metricas_c['comentarios'].append(interacao['comment_text'])

    # Calcular média de visualização final
    for id_c in metricas_conteudo:
        metricas_c = metricas_conteudo[id_c] # Acessa as métricas do conteúdo pelo ID
        if metricas_c['contagem_watch_duration_para_media'] > 0:
            metricas_c['media_tempo_visualizacao'] = round(metricas_c['soma_watch_duration_para_media'] / metricas_c['contagem_watch_duration_para_media'], 3) # Calcula a média do tempo de visualização, com 2 casas decimais
        # Remove campo auxiliar da soma
        if 'soma_watch_duration_para_media' in metricas_c:
            del metricas_c['soma_watch_duration_para_media']
        # Remove campo auxiliar da contagem
        if 'contagem_watch_duration_para_media' in metricas_c:
            del metricas_c['contagem_watch_duration_para_media']

    return dict(metricas_conteudo) # Converter de volta para dict normal para exibição


def main():
    """
    Função principal para orquestrar a análise.
    """
    print("Iniciando Fase 1: Coleta e Estruturação Inicial de Dados de Engajamento Globo\n")

    # 1. Carregar dados do arquivo CSV para uma lista de listas
    dados_brutos_lista_de_listas = carregar_dados_de_arquivo_csv(NOME_ARQUIVO_CSV)

    if dados_brutos_lista_de_listas is None:
        print(f"Não foi possível carregar os dados do arquivo '{NOME_ARQUIVO_CSV}'. Encerrando.")
        return

    if len(dados_brutos_lista_de_listas) < 2: # Precisa de cabeçalho + pelo menos uma linha de dados
        print(f"O arquivo '{NOME_ARQUIVO_CSV}' não contém dados suficientes (cabeçalho e linhas de dados). Encerrando.")
        return

    print(f"Total de {len(dados_brutos_lista_de_listas) - 1} linhas de dados (mais cabeçalho) carregadas do CSV.\n")

    # ETAPA PARA OS ALUNOS: Converter a lista de listas para uma lista de dicionários
    # Esta etapa é crucial para que as funções subsequentes funcionem como esperado.
    interacoes_brutas_dict = converter_lista_para_lista_de_dicionarios(dados_brutos_lista_de_listas)


    # 2. Limpar e transformar dados (agora a partir da lista de dicionários)
    interacoes_limpas = limpar_e_transformar_dados(interacoes_brutas_dict)


    # 3. Criar mapa de conteúdos (id_conteudo -> nome_conteudo)
    mapa_conteudos = criar_mapa_conteudos(interacoes_limpas)

    # 4. Calcular métricas por conteúdo
    metricas = calcular_metricas_por_conteudo(interacoes_limpas, mapa_conteudos)


    # 5. Exibir resultados
    for id_conteudo, dados in metricas.items():
        print(f"\nConteúdo: {dados['nome_conteudo']}")
        print(f"Total de engajamento (like/share/comment/vote_bbb): {dados['total_interacoes_engajamento']}")
        print("Contagem por tipo de interação:")
        for tipo, qtd in dados['contagem_por_tipo_interacao'].items():
            print(f"  - {tipo}: {qtd}")
        print(f"Tempo total de visualização: {dados['tempo_total_visualizacao']} segundos")
        print(f"Média de tempo assistido: {dados['media_tempo_visualizacao']} segundos")
        print(f"Comentários ({len(dados['comentarios'])}):")
        for c in dados['comentarios']:
            print(f"   • {c}")

# Executa a função principal se este script for executado diretamente
if __name__ == "__main__":
    main()

Iniciando Fase 1: Coleta e Estruturação Inicial de Dados de Engajamento Globo

Total de 187 linhas de dados (mais cabeçalho) carregadas do CSV.


Conteúdo: Jornal Nacional
Total de engajamento (like/share/comment/vote_bbb): 0
Contagem por tipo de interação:
  - view_start: 10
Tempo total de visualização: 18120 segundos
Média de tempo assistido: 1812.0 segundos
Comentários (0):

Conteúdo: Novela Renascer
Total de engajamento (like/share/comment/vote_bbb): 4
Contagem por tipo de interação:
  - view_start: 6
  - like: 1
  - comment: 1
  - share: 2
Tempo total de visualização: 16290 segundos
Média de tempo assistido: 2715.0 segundos
Comentários (1):
   • Que capítulo emocionante!

Conteúdo: Podcast Papo de Segunda
Total de engajamento (like/share/comment/vote_bbb): 14
Contagem por tipo de interação:
  - view_start: 7
  - comment: 9
  - share: 3
  - like: 2
Tempo total de visualização: 24850 segundos
Média de tempo assistido: 3550.0 segundos
Comentários (9):
   • Muito bom o tema de hoje!
 